# Тестовый файл для удобного запуска и отладки составных частей бота 

In [54]:
%reload_ext autoreload
%autoreload 2

In [55]:
import os
from dotenv import load_dotenv
from tinkoff.invest import Client

from tinkoff.invest import CandleInterval
from datetime import datetime

load_dotenv()

TOKEN = os.getenv("INVEST_TOKEN")

TICKER = 'RNFT'
FIGI = 'BBG00F9XX7H4'

In [56]:
from trader_bot import ScalpingBot

In [57]:
year, month, day, tmz = 2024, 3, 15, 3

date_from = datetime(year, month, day, 9 - tmz, 55)
date_to = datetime(year, month, day, 19 - tmz, 0)

print("Начальная дата и время:", date_from)
print("Конечная дата и время:", date_to)

Начальная дата и время: 2024-03-15 06:55:00
Конечная дата и время: 2024-03-15 16:00:00


In [58]:
with Client(TOKEN) as client:
    candles_1_min = client.market_data.get_candles(
        figi=FIGI,
        from_=date_from,
        to=date_to,
        interval=CandleInterval.CANDLE_INTERVAL_1_MIN
    )
    
    candles_5_min = client.market_data.get_candles(
        figi=FIGI,
        from_=date_from,
        to=date_to,
        interval=CandleInterval.CANDLE_INTERVAL_5_MIN
    )

# Тестовая система

In [64]:
from datetime import timedelta
from test_env.client_test_env import ClientTestEnvHelper
from test_env.logger_test_env import LoggerTestEnvHelper
from test_env.time_test_env import TimeTestEnvHelper
from test_env.database_test_env import DatabaseTestEnvHelper

time_helper = TimeTestEnvHelper(date_from)
logger_helper = LoggerTestEnvHelper(time_helper)
client_helper = ClientTestEnvHelper(TICKER, logger_helper, time_helper, candles_1_min)
database_helper = DatabaseTestEnvHelper(client_helper)
# todo вот это пока задается костылем  
client_helper.set_ticker_params(1, 0.1, 'BBG00F9XX7H4', 'RUR')

# ---------------

bot = ScalpingBot(
    TOKEN, 
    TICKER,
    time_helper=time_helper,
    logger_helper=logger_helper,
    client_helper=client_helper,
    database_helper=database_helper,
    )

# итератор по минутам для заданного времени
def datetime_iterator(start_datetime, end_datetime):
    current_datetime = start_datetime
    while current_datetime <= end_datetime:
        yield current_datetime
        current_datetime += timedelta(minutes=1)

bot.time.set_time(date_from)
bot.time.sleep(0)

# Использование итератора для вывода каждой пары час-минута
for dt in datetime_iterator(date_from, date_to):
    # задаем время
    time_helper.set_time(dt)
    
    candle = client_helper.get_candle(dt)
    if candle is None:
        continue
    
    # задаем текущее значение свечи
    client_helper.set_current_candle(candle)
    
    # анализируем заявки
    if bot.buy_order:
        buy_price = client_helper.quotation_to_float(bot.buy_order.initial_order_price)
        low_buy_price = client_helper.quotation_to_float(candle.low)
        if buy_price >= low_buy_price:
            client_helper.buy_order_executed = True
            client_helper.buy_order_executed_on_border = buy_price == low_buy_price

    elif bot.sell_order:
        sell_price = client_helper.quotation_to_float(bot.sell_order.initial_order_price)
        high_sell_price = client_helper.quotation_to_float(candle.high)
        if sell_price <= high_sell_price:
            client_helper.sell_order_executed = True
            client_helper.sell_order_executed_on_border = sell_price == high_sell_price
    
        # успешные помечаем  

    # todo допиши сначала выборку свечи, если она не полная еще  
    
    # если пора просыпаться
    if time_helper.is_time_to_awake():
        # print(dt.strftime("%H:%M"))
        # запускаем итерацию торгового алгоритма
        bot.run_iteration()

bot.stop()

print()
print(f"Изменение баланса: {round(database_helper.sum, 2)} {client_helper.currency}")
print(f"Операций: {client_helper.total_completed_orders}, "
      f"из них граничных {client_helper.total_completed_orders_on_border}"
      f"({round(client_helper.total_completed_orders_on_border / client_helper.total_completed_orders, 2)}%)")

9:55 - INIT 
     figi - BBG00F9XX7H4 (RNFT)
     candles_count - 4
     min profit - 5 steps * 0.1 = 0.5 RUR
     stop_loss_percent - 1.0 %
     commission - 0.05 %
     no_operation_timeout_seconds - 300 sec
     sleep_trading - 420 sec
     sleep_no_trade - 420 sec

9:59 - can not trade, sleep 300
.......10:34 - 5.0 минут без активности. Снимаем и переставляем заявки.
10:34 - Размещена заявка на покупку по 173.7
10:34 - Ждем следующего цикла, sleep 420
10:41 - 5.0 минут без активности. Снимаем и переставляем заявки.
10:41 - Buy order 111, price=173.7 canceled
10:41 - Размещена заявка на покупку по 174.3
10:41 - Ждем следующего цикла, sleep 420
10:48 - BUY order executed, price 174.3
10:48 - Пока не торгуем. Ожидаемая разница в торгах - 0.4, а требуется минимум 0.5 
10:48 - Ждем следующего цикла, sleep 420
10:55 - 5.0 минут без активности. Снимаем и переставляем заявки.
10:55 - Пока не торгуем. Ожидаемая разница в торгах - 0.4, а требуется минимум 0.5 
10:55 - Ждем следующего цикла, 